In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# 제품 데이터 저장 리스트
products_data = []

# Selenium 드라이버 설정
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    # Selenium 탐지 우회
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 제품 정보 추출 함수
def extract_product_info(product, driver):
    try:
        if product.get("id", "").startswith("adPointProductItem") or "adSmartAreaTop" in product.get("id", ""):
            return False

        photo_elem = product.select_one(".thumb_image img")
        if photo_elem:
            photo = photo_elem.get("data-original") or photo_elem.get("src") or "N/A"
            if "noImg_130.gif" in photo:
                photo = "N/A"
        else:
            photo = "N/A"

        name_elem = product.select_one(".prod_name a")
        name = name_elem.text.strip() if name_elem else "Unknown"
        detail_url = name_elem["href"] if name_elem and "href" in name_elem.attrs else None

        # 상세 페이지에서 spec 수집
        spec_dict = {}
        if detail_url:
            # 현재 탭의 핸들 저장
            main_window = driver.current_window_handle

            # 새 탭 열기 (JavaScript 실행)
            driver.execute_script("window.open('', '_blank');")

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 새 탭에서 상세 페이지 열기
            driver.get(detail_url)
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_spec table"))
                )
            except:
                print(f"❗️스펙 테이블 로딩 실패 (대기 시간 초과): {name}")

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            spec_table = detail_soup.select_one(".prod_spec .spec_tbl tbody") or detail_soup.select_one(".prod_spec table")
            if spec_table:
                for row in spec_table.find_all("tr"):
                    keys = row.find_all("th", class_="tit")
                    values = row.find_all("td", class_="dsc")
                    for key_elem, value_elem in zip(keys, values):
                        key = key_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)
                        spec_dict[key] = value

            # 탭 닫기
            driver.close()

            # 원래 탭으로 돌아가기
            driver.switch_to.window(main_window)

        price_elem = product.select_one(".price_sect a")
        price = price_elem.text.strip().replace(",", "") if price_elem else "N/A"

        rating_elem = product.select_one(".star-single .text__score") or product.select_one(".point_num")
        rating = rating_elem.text.strip() if rating_elem else "N/A"

        product_info = {
            "url": photo,
            "Name": name,
            "Price": price,
            "Rating": rating,
            **spec_dict
        }
        products_data.append(product_info)
        print(f"수집: {name}, 사진: {photo}, 가격: {price}, 평점: {rating}")
        return True

    except Exception as e:
        print(f"제품 정보 수집 오류: {e}")

        # 혹시 새 탭이 열려있는 상태라면 닫고 메인 탭으로 돌아가기
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass

        return False

# 페이지 스크레이핑 함수
def scrape_page(driver, page_num):
    try:
        # 현재 페이지의 제품 정보 수집
        soup = BeautifulSoup(driver.page_source, "html.parser")
        products = soup.select(".prod_item.prod_layer")
        print(f"페이지 {page_num}에서 찾은 제품 수: {len(products)}")

        if not products:
            print("더 이상 제품이 없습니다.")
            return False

        for product in products:
            if product.find_parent(id="adPointArea") or product.find_parent(id="adSmartAreaTop"):
                continue
            extract_product_info(product, driver)

        return True
    except Exception as e:
        print(f"페이지 수집 오류 (페이지 {page_num}): {e}")
        return False

# 크롤링 함수
def crawl_danawa_cpu_to_dataframe(max_pages=None):
    driver = setup_driver()
    try:
        base_url = "https://prod.danawa.com/list/?cate=112752"
        driver.get(base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기

        page_num = 1
        while True:
            if max_pages and page_num > max_pages:
                break

            print(f"페이지 {page_num} 수집 중...")

            # 현재 페이지 수집
            scrape_page(driver, page_num)

            if max_pages and page_num >= max_pages:
                break

            # 다음 페이지로 이동
            try:
                # 페이지 이동 방식 결정
                if page_num % 10 == 0:  # 10, 20, 30 등 10의 배수 페이지일 경우
                    # 다음 페이지 그룹 버튼 클릭
                    next_page_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.edge_nav.nav_next"))
                    )
                    next_page_button.click()
                else:
                    # 일반 페이지 이동
                    driver.execute_script(f"movePage({page_num + 1}); return false;")

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_item.prod_layer"))
                )
                time.sleep(2)  # 추가 대기 시간

                page_num += 1
            except Exception as e:
                print(f"다음 페이지로 이동 실패: {e}")
                break

        df = pd.DataFrame(products_data)
        print(f"총 {len(products_data)}개 CPU 제품 수집 완료")
        return df

    finally:
        driver.quit()
        print("Selenium 드라이버 종료")

# 실행
max_pages = 31  # 테스트를 위해 작은 값으로 설정
df = crawl_danawa_cpu_to_dataframe(max_pages=max_pages)
print("\n수집된 데이터프레임 미리보기:")
print(df.head())
df.to_csv("danawa_ram_products.csv", index=False, encoding="utf-8-sig")
print("데이터가 'danawa_ram_products.csv' 파일로 저장되었습니다.")

페이지 1 수집 중...
페이지 1에서 찾은 제품 수: 30
수집: TeamGroup DDR5-5600 CL46 Elite 서린, 사진: //img.danawa.com/prod_img/500000/839/535/img/17535839_1.jpg?shrink=130:130&_v=20220726132209, 가격: 120390원, 평점: 4.8
수집: ESSENCORE KLEVV DDR4-3200 CL22 파인인포, 사진: //img.danawa.com/prod_img/500000/091/787/img/11787091_1.jpg?shrink=130:130&_v=20241022144728, 가격: 78000원, 평점: 4.8
수집: 삼성전자 DDR4-3200, 사진: //img.danawa.com/prod_img/500000/199/790/img/11790199_1.jpg?shrink=130:130&_v=20250418095009, 가격: 98400원, 평점: 4.7
수집: ESSENCORE KLEVV DDR5-5600 CL46 파인인포, 사진: //img.danawa.com/prod_img/500000/774/965/img/18965774_1.jpg?shrink=130:130&_v=20240911124521, 가격: 125270원, 평점: 4.7
수집: CORSAIR DDR5-6000 CL30 VENGEANCE RGB DP WHITE 패키지, 사진: //img.danawa.com/prod_img/500000/092/471/img/72471092_1.jpg?shrink=130:130&_v=20241206111238, 가격: 178990원, 평점: N/A
수집: 마이크론 Crucial DDR4-3200 CL22 대원씨티에스, 사진: //img.danawa.com/prod_img/500000/935/805/img/35805935_1.jpg?shrink=130:130&_v=20240214094800, 가격: 39490원, 평점: 5.0
수집: 마이크론 Crucial DD

In [4]:
import pandas as pd
df = pd.read_csv("danawa_ram_products.csv", encoding="UTF-8")
df

,url,Name,Price,Rating,제조회사,등록년월,사용 장치,제품 분류,메모리 규격,메모리 용량,...,클럭드라이버,XMP,RGB SYNC,ECC,REG,유통회사,RAZER CHROMA,TT RGB PLUS,T-FORCE BLITZ,G.SKILL Lighting
0,//img.danawa.com/prod_img/500000/839/535/img/1...,TeamGroup DDR5-5600 CL46 Elite 서린,120390원,4.8,TeamGroup(제조사 웹사이트 바로가기),2022년 07월,데스크탑용,DDR5,DIMM(UDIMM),16GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,//img.danawa.com/prod_img/500000/091/787/img/1...,ESSENCORE KLEVV DDR4-3200 CL22 파인인포,78000원,4.8,ESSENCORE(제조사 웹사이트 바로가기),2020년 07월,데스크탑용,DDR4,DIMM(UDIMM),16GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,//img.danawa.com/prod_img/500000/199/790/img/1...,삼성전자 DDR4-3200,98400원,4.7,삼성전자(제조사 웹사이트 바로가기),2020년 07월,데스크탑용,DDR4,DIMM(UDIMM),16GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,//img.danawa.com/prod_img/500000/774/965/img/1...,ESSENCORE KLEVV DDR5-5600 CL46 파인인포,125270원,4.7,ESSENCORE(제조사 웹사이트 바로가기),2023년 02월,데스크탑용,DDR5,DIMM(UDIMM),16GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,//img.danawa.com/prod_img/500000/092/471/img/7...,CORSAIR DDR5-6000 CL30 VENGEANCE RGB DP WHITE 패키지,178990원,NaN,CORSAIR(제조사 웹사이트 바로가기),2024년 12월,데스크탑용,DDR5,DIMM(UDIMM),32GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,//img.danawa.com/prod_img/500000/443/794/img/7...,CYNEX DDR3-1333 엠트란,26150원,NaN,CYNEX,2008년 11월,데스크탑용,DDR3,DIMM(UDIMM),2GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,//img.danawa.com/prod_img/500000/964/793/img/7...,CYNEX DDR3-1066 멜론,24000원,NaN,CYNEX,2008년 11월,데스크탑용,DDR3,DIMM(UDIMM),2GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901,//img.danawa.com/prod_img/500000/961/793/img/7...,CYNEX DDR3-1066 엠트란,26000원,NaN,CYNEX,2008년 11월,데스크탑용,DDR3,DIMM(UDIMM),2GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,//img.danawa.com/prod_img/500000/959/793/img/7...,CYNEX DDR3-1066,25220원,NaN,CYNEX,2008년 11월,데스크탑용,DDR3,DIMM(UDIMM),2GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# c 컬럼의 인덱스 찾기
index = df.columns.get_loc('적합성평가인증')

# c 컬럼 좌측의 컬럼들 선택
left_columns = df.columns[:index]
df = df[left_columns]

# 공백을 기준으로 첫 번째 단어 추출
df['등록년'] = df['등록년월'].str.split().str[0]
df['등록월'] = df['등록년월'].str.split().str[1]
df['제조회사'] = df['제조회사'].str.split('\(').str[0]

df.drop(['등록년월', 'Unnamed: 15'], axis=1, inplace=True)

df

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
C:\Users\ms0\AppData\Local\Temp\ipykernel_8704\2303062987.py:11: SyntaxWarning: invalid escape sequence '\('
  df['제조회사'] = df['제조회사'].str.split('\(').str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_8704\2303062987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록년'] = df['등록년월'].str.split().str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_8704\2303062987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,url,Name,Price,Rating,제조회사,사용 장치,제품 분류,메모리 규격,메모리 용량,동작클럭(대역폭),램타이밍,동작전압,램개수,온다이ECC,히트싱크,높이,등록년,등록월
0,//img.danawa.com/prod_img/500000/839/535/img/1...,TeamGroup DDR5-5600 CL46 Elite 서린,120390원,4.8,TeamGroup,데스크탑용,DDR5,DIMM(UDIMM),16GB,5600MHz (PC5-44800),CL46-46-46-90,1.10V,1개,○,미포함,32mm,2022년,07월
1,//img.danawa.com/prod_img/500000/091/787/img/1...,ESSENCORE KLEVV DDR4-3200 CL22 파인인포,78000원,4.8,ESSENCORE,데스크탑용,DDR4,DIMM(UDIMM),16GB,3200MHz (PC4-25600),CL22-22-22-51,1.20V,1개,NaN,미포함,NaN,2020년,07월
2,//img.danawa.com/prod_img/500000/199/790/img/1...,삼성전자 DDR4-3200,98400원,4.7,삼성전자,데스크탑용,DDR4,DIMM(UDIMM),16GB,3200MHz (PC4-25600),NaN,NaN,1개,NaN,미포함,NaN,2020년,07월
3,//img.danawa.com/prod_img/500000/774/965/img/1...,ESSENCORE KLEVV DDR5-5600 CL46 파인인포,125270원,4.7,ESSENCORE,데스크탑용,DDR5,DIMM(UDIMM),16GB,5600MHz (PC5-44800),CL46-46-46-90,1.10V,1개,○,미포함,31.25mm,2023년,02월
4,//img.danawa.com/prod_img/500000/092/471/img/7...,CORSAIR DDR5-6000 CL30 VENGEANCE RGB DP WHITE 패키지,178990원,NaN,CORSAIR,데스크탑용,DDR5,DIMM(UDIMM),32GB,6000MHz (PC5-48000),CL30-36-36-76,1.40V,2개,NaN,방열판,56mm,2024년,12월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,//img.danawa.com/prod_img/500000/443/794/img/7...,CYNEX DDR3-1333 엠트란,26150원,NaN,CYNEX,데스크탑용,DDR3,DIMM(UDIMM),2GB,1333MHz (PC3-10600),CL9,NaN,1개,NaN,방열판,NaN,2008년,11월
900,//img.danawa.com/prod_img/500000/964/793/img/7...,CYNEX DDR3-1066 멜론,24000원,NaN,CYNEX,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,NaN,1개,NaN,방열판,NaN,2008년,11월
901,//img.danawa.com/prod_img/500000/961/793/img/7...,CYNEX DDR3-1066 엠트란,26000원,NaN,CYNEX,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,NaN,1개,NaN,방열판,NaN,2008년,11월
902,//img.danawa.com/prod_img/500000/959/793/img/7...,CYNEX DDR3-1066,25220원,NaN,CYNEX,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,1.50V,1개,NaN,미포함,NaN,2008년,11월


In [9]:
df.to_csv("ram.csv", index=False, encoding="utf-8-sig")

In [3]:
df.head().to_csv("ram_sample.csv", index=False, encoding="utf-8-sig")

In [26]:
# 1. 첫 번째 테이블: idx (1000부터 시작)와 a, b 컬럼
ram_product_df = pd.DataFrame({
    'product_idx': range(2868, 2868 + len(df)),
    'name': df['Name'],
    'brand': df['제조회사'],
    'price': df['Price'],
    'rating': df['Rating'],
    'category': "RAM"
})

# 2. 두 번째 테이블: idx (index+1), table1의 idx를 외래키로, c 컬럼
ram_img_df = pd.DataFrame({
    'product_image_idx': range(2868, 2868 + len(df)),
    'product_idx': ram_product_df['product_idx'],  # table1의 idx를 외래키로
    'img_url': df['url']
})

used_columns = ['Name', 'Price', 'url', 'Rating', '제조회사']
# 원래 데이터프레임에서 사용되지 않은 컬럼 선택
remaining_columns = [col for col in df.columns if col not in used_columns]

# table3 생성
ram_detail_df = pd.DataFrame({
    'ram_idx': range(1, len(df) + 1),
    'product_idx': ram_product_df['product_idx']
})
# 나머지 컬럼 추가
for col in remaining_columns:
    ram_detail_df[col] = df[col]

In [27]:
ram_img_df.to_csv("ram_img.csv", index=False, encoding="utf-8-sig")

In [28]:
ram_product_df.to_csv("ram_product.csv", index=False, encoding="utf-8-sig")

In [31]:
ram_detail_df.to_csv("ram_detail.csv", index=False, encoding="utf-8-sig")

In [29]:
ram_detail_df.drop(['온다이ECC'], axis=1, inplace=True)

In [30]:
ram_detail_df

,ram_idx,product_idx,사용 장치,제품 분류,메모리 규격,메모리 용량,동작클럭(대역폭),램타이밍,동작전압,램개수,히트싱크,높이,등록년,등록월
0,1,2868,데스크탑용,DDR5,DIMM(UDIMM),16GB,5600MHz (PC5-44800),CL46-46-46-90,1.10V,1개,미포함,32mm,2022년,07월
1,2,2869,데스크탑용,DDR4,DIMM(UDIMM),16GB,3200MHz (PC4-25600),CL22-22-22-51,1.20V,1개,미포함,NaN,2020년,07월
2,3,2870,데스크탑용,DDR4,DIMM(UDIMM),16GB,3200MHz (PC4-25600),NaN,NaN,1개,미포함,NaN,2020년,07월
3,4,2871,데스크탑용,DDR5,DIMM(UDIMM),16GB,5600MHz (PC5-44800),CL46-46-46-90,1.10V,1개,미포함,31.25mm,2023년,02월
4,5,2872,데스크탑용,DDR5,DIMM(UDIMM),32GB,6000MHz (PC5-48000),CL30-36-36-76,1.40V,2개,방열판,56mm,2024년,12월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,900,3767,데스크탑용,DDR3,DIMM(UDIMM),2GB,1333MHz (PC3-10600),CL9,NaN,1개,방열판,NaN,2008년,11월
900,901,3768,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,NaN,1개,방열판,NaN,2008년,11월
901,902,3769,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,NaN,1개,방열판,NaN,2008년,11월
902,903,3770,데스크탑용,DDR3,DIMM(UDIMM),2GB,1066MHz (PC3-8500),CL7,1.50V,1개,미포함,NaN,2008년,11월
